In [ ]:
# randomgrowth.jl
using Interact
using Compose

set_default_graphic_size(10Compose.cm,10Compose.cm)
# representation: array of points representing cells
# which could grow in the next iteration

In [ ]:
function draw_cg(pts, dim, grew)
    x, y = transpose(first.(pts)), transpose(last.(pts))
    x_g, y_g = x[grew], y[grew]
    bound, unit = dim .+ 0*x, 1 .+ 0*x
    
    highlights = nothing
    if length(x_g) > 0
        highlights = (context(), rectangle(x_g, y_g, unit, unit), fill("yellow"))
    end
    return compose(context(units=UnitBox(0, 0, dim, dim)),
            highlights,
            (context(), rectangle(x, y, bound, bound), fill("black")))
end

In [ ]:
function append(arr, pt)
    if arr[end][1] == pt[1] || arr[end][2] == pt[2]
        arr[end] = (min(arr[end][1], pt[1]), min(arr[end][2], pt[2]))
    else
        push!(arr, pt)
    end
end

function extend(pts, grew)
    # ret = [(-1, Inf)]
    ret = [(-1, 5 * (5 + maximum(hcat(first.(pts), last.(pts)))))]
    
    for i = 1:length(grew)
        if grew[i]
            append(ret, (pts[i][1], pts[i][2] + 1))
            append(ret, (pts[i][1] + 1, pts[i][2]))
        else
            append(ret, (pts[i][1], pts[i][2]))
        end
    end
    return ret[2:end]
end

In [ ]:
function calculate_dynamics(N, q, T=20, highlight=false)
    cur = [(0, 1), (1, 0)]
    all_images = [draw_cg(cur, N, falses(size(cur)))]
    for i=1:T
        grew = BitArray(rand(Bool, size(cur)))
        if highlight
            push!(all_images, draw_cg(cur, N, trues(size(cur))))
            push!(all_images, draw_cg(cur, N, grew))
        end
        
        cur = extend(cur, grew)
        push!(all_images, draw_cg(cur, N, falses(size(cur))))
    end
    return all_images
end

function visualize_dynamics(all_images)
    @manipulate for i in slider(1:length(all_images), value=1)
    #for i in 1:length(all_images)
        all_images[i]
    end
end

In [ ]:
visualize_dynamics(calculate_dynamics(10000, 0.5, 4000))

In [ ]:
# analytical limiting shape:
limiting(x, y) = x + y + 2*√(x*y*q)